In [1]:
import os
import sys
os.environ['PATH']+=':'+sys.path[1].split('lib')[0]+'bin'
sys.path.append('/home/diego/Projects')

import tempfile
import numpy as np

import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app

from openmmtools import states, mcmc

from yank.multistate import ParallelTemperingSampler, MultiStateReporter, ParallelTemperingAnalyzer

import matplotlib.pyplot as plt

import TestSystems

plt.style.use("ggplot")

In [2]:
working_system = TestSystems.TrpCageImplicit()

In [3]:
n_replicas = 8
T_min      = 298.0*unit.kelvin
T_max      = 600.0*unit.kelvin

In [4]:
reference_state = states.ThermodynamicState(system=working_system.system, temperature=T_min)

In [5]:
move = mcmc.GHMCMove(timestep=2.0*unit.femtoseconds, n_steps=50)

In [6]:
simulation = ParallelTemperingSampler(mcmc_moves=move, number_of_iterations=500)

In [7]:
storage_path = tempfile.NamedTemporaryFile(delete=False).name + '.nc'
reporter = MultiStateReporter(storage_path, checkpoint_interval=10)

In [8]:
simulation.create(reference_state, states.SamplerState(working_system.positions), reporter,
                  min_temperature=T_min, max_temperature=T_max, n_temperatures=n_replicas)

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

In [9]:
simulation.run(n_iterations=15000) #sera posible poner tqdm en las replicas?

KeyboardInterrupt: 

In [ ]:
analysis=ParallelTemperingAnalyzer(reporter)

In [ ]:
analysis.generate_mixing_statistics()

In [10]:
os.remove(storage_path)